<a href="https://colab.research.google.com/github/NGJIERU/UMHack2025/blob/v2/DAXAssistant%E2%80%93Handsfree-V2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -qq portaudio19-dev python-all-dev python3-all-dev
!pip install pyaudio
!pip install SpeechRecognition
!pip install pyttsx3

In [ ]:
!pip install pydub ffmpeg-python google-cloud-texttospeech

In [ ]:
!pip install gTTS


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving daxassistant-handsfree-a732640a27d5.json to daxassistant-handsfree-a732640a27d5 (1).json


In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "daxassistant-handsfree-a732640a27d5.json"


In [ ]:
import IPython
import ipywidgets as widgets
import base64
import io
import time
import numpy as np
from pydub import AudioSegment
from IPython.display import Audio, display
from google.colab import output  # ✅ Needed for callback

# Create a hidden widget to store audio
audio_data_widget = widgets.Text()
display(audio_data_widget)

# JavaScript to record audio from mic
record_js = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader();
  reader.onloadend = e => resolve(reader.result);
  reader.readAsDataURL(blob);
});

var record = async () => {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const recorder = new MediaRecorder(stream);
  const chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();
  await sleep(5000);
  recorder.stop();

  await new Promise(resolve => recorder.onstop = resolve);
  const blob = new Blob(chunks);
  const b64 = await b2text(blob);
  google.colab.kernel.invokeFunction('notebook.recordedAudio', [b64], {});
};
record();
"""

# Callback function to receive data from JS
def _handle_audio(js_data):
    audio_data_widget.value = js_data

# ✅ Use google.colab.output instead of IPython
output.register_callback('notebook.recordedAudio', _handle_audio)

# Run the JS code
IPython.display.display(IPython.display.Javascript(record_js))


Text(value='')

<IPython.core.display.Javascript object>

In [ ]:
# Wait until audio is recorded
while audio_data_widget.value == "":
    print("Waiting for recording to finish...")
    time.sleep(1)

# Decode base64 to audio bytes
audio_bytes = base64.b64decode(audio_data_widget.value.split(',')[1])
audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format="webm")

# Convert to 16kHz mono NumPy array
audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
samples = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0

# Play the recorded audio
print("🎙️ Playing your recording:")
display(Audio(samples, rate=16000))


🎙️ Playing your recording:


In [ ]:
from google.cloud import texttospeech

# Set up Google Cloud TTS client
client = texttospeech.TextToSpeechClient()

# Define the text response
text = "Okay, attempting to reroute."

# Configure voice parameters
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US",
    name="en-US-Wavenet-D"
)

# Configure output audio format
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.LINEAR16
)

# Synthesize speech
response = client.synthesize_speech(
    request={
        "input": {"text": text},
        "voice": voice,
        "audio_config": audio_config
    }
)

# Play TTS result
print("🗣️ TTS Response:")
display(Audio(response.audio_content, rate=16000))


PermissionDenied: 403 Cloud Text-to-Speech API has not been used in project 896231400660 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/texttospeech.googleapis.com/overview?project=896231400660 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "texttospeech.googleapis.com"
}
metadata {
  key: "serviceTitle"
  value: "Cloud Text-to-Speech API"
}
metadata {
  key: "containerInfo"
  value: "896231400660"
}
metadata {
  key: "consumer"
  value: "projects/896231400660"
}
metadata {
  key: "activationUrl"
  value: "https://console.developers.google.com/apis/api/texttospeech.googleapis.com/overview?project=896231400660"
}
, locale: "en-US"
message: "Cloud Text-to-Speech API has not been used in project 896231400660 before or it is disabled. Enable it by visiting https://console.developers.

In [ ]:
import librosa
import numpy as np
from IPython.display import Audio, display

def record_audio(duration=5):
    print(f"Recording for {duration} seconds...")
    audio = librosa.tone(frequency=440, sr=16000, length=duration * 16000) # Placeholder
    # In a real scenario, you'd use a widget or browser interaction to capture audio
    print("Recording finished.")
    return audio

recorded_audio = record_audio()
display(Audio(recorded_audio, rate=16000))

Recording for 5 seconds...
Recording finished.


In [ ]:
from google.cloud import texttospeech
from IPython.display import Audio, display

# Your Google Cloud credentials setup here

client = texttospeech.TextToSpeechClient()
text = "Okay, attempting to reroute."
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US",
    name="en-US-Wavenet-D",
)
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.LINEAR16
)
response = client.synthesize_speech(
    request={"input": {"text": text}, "voice": voice, "audio_config": audio_config}
)
display(Audio(response.audio_content, rate=16000))

ImportError: cannot import name 'texttospeech' from 'google.cloud' (unknown location)